In [1]:
import numpy as np
import pandas as pd
import scipy.stats

pd.options.mode.chained_assignment = None
from matplotlib import pyplot as plt, rcParams
# import cv2
import seaborn as sns

sns.set(style="white", context="paper")
from cycler import cycler
import os, sys
import glob
from datetime import datetime, timedelta
from itertools import combinations
import base64
from PIL import Image
from io import BytesIO as _BytesIO
import requests
import json
import pickle
from datetime import datetime
from IPython.display import display, Markdown, Latex
from sklearn.metrics import *
import collections
from copy import deepcopy
import traceback
from sympy import Point, Polygon
import mmcv
import cv2
from mmtrack.apis import inference_mot, init_model as init_tracking_model
from mmpose.apis import inference_top_down_pose_model, init_pose_model, vis_pose_result
from xtcocotools.coco import COCO
from mmpose.datasets import DatasetInfo
# import plotly
# from pandas_profiling import ProfileReport

pd.options.display.max_columns = None


def printm(s): return display(Markdown(s))

/home/prasoon/anaconda3/envs/edusense/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/prasoon/anaconda3/envs/edusense/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
'''
In this notebook, we wish to show visualization of id_tracking changes, specially in places where we updated ids to prove that id switches were successful.
To do this, We start with finding frame numbers to trim different sized videos. 

Start with creating id matches based on start/stop vicinity and bb-tracking. For each match, create a different video from min_frame-50, max_frame+50. Loop over video and capture relevant frames. 


'''
# !pip install -U numba

'\nIn this notebook, we wish to show visualization of id_tracking changes, specially in places where we updated ids to prove that id switches were successful.\nTo do this, We start with finding frame numbers to trim different sized videos. \n\nStart with creating id matches based on start/stop vicinity and bb-tracking. For each match, create a different video from min_frame-50, max_frame+50. Loop over video and capture relevant frames. \n\n\n'

In [3]:
course = '05410A'
session = 'classinsight-cmu_05410A_ghc_4301_201902181500'
session_id_file = f'/mnt/ci-nas-cache/edulyzeV2/tracking_analysis/session_tracking_info/{session}-front.pb'
session_frame_dir = f'/mnt/ci-nas-cache/edulyzeV2/track/{course}/{session}-front'
session_video_file = f'/mnt/ci-nas-classes/classinsight/2019S/video_backup/{session}/{session}-front.avi'
assert os.path.exists(session_video_file)
# config for single session tracking
MIN_ID_FRAMES = 900
MAX_ID_DISTANCE = 900
MAX_BBOX_OVERLAP = 0.4
print(len(glob.glob(f"{session_frame_dir}/*")))

23844


In [4]:

# get tracking data
df_tracking = pickle.load(open(session_id_file,'rb')).transpose()
df_tracking.head()
print("Got Tracking Data...")

# Get id start and stop for raw tracking data
col_start_stop_idxs = []
MIN_ID_FRAMES = 900
MAX_ID_DISTANCE = 900
MAX_BBOX_OVERLAP = 0.4
for col in df_tracking.columns:
    one_idxs = df_tracking.index[np.where(df_tracking[col]==1)[0]].values
    col_start_stop_idxs.append([col, one_idxs.min(), one_idxs.max()])
df_id_start_stop = pd.DataFrame(col_start_stop_idxs, columns=['id','min_idx','max_idx'])
df_id_start_stop['total_idxs'] = df_id_start_stop['max_idx']-df_id_start_stop['min_idx']
df_id_start_stop = df_id_start_stop[df_id_start_stop.total_idxs>MIN_ID_FRAMES].reset_index(drop=True)
total_idxs = df_id_start_stop.max_idx.max()
print("Got ID start and stop from raw data...")


# get potential id maps
final_id_maps = {}
direct_id_maps = {}
final_matches = []
for row_idx, row in df_id_start_stop.iterrows():
    row_maxidx = row['max_idx']
    
    # get polygon for given id
    id_max_frame = row_maxidx
    id_frame_data = pickle.load(open(f"{session_frame_dir}/{id_max_frame}.pb","rb"))[1]
    id_frame_data = [xr for xr in id_frame_data if (xr['track_id']==row['id'])][0]
    id_bb = id_frame_data['bbox'][:4].astype(int)
    X_TL1, Y_TL1, X_BR1, Y_BR1 = id_bb
    # area_id_bb = (X_BR1-X_TL1)* (Y_BR1-Y_TL1)
    p1, p2, p3, p4  = map(Point, [[X_TL1,Y_TL1], [X_TL1,Y_BR1], [X_BR1,Y_BR1],[X_BR1, Y_TL1]]) 
    id_polygon = Polygon(p1, p2, p3, p4)

    potential_id_matches = df_id_start_stop[(df_id_start_stop.min_idx<=row_maxidx+MAX_ID_DISTANCE) & (df_id_start_stop.min_idx>row_maxidx-MAX_ID_DISTANCE)].id.values
    successful_matches = []
    if len(potential_id_matches)>0:
        for matched_id in potential_id_matches:
            
            matched_id_min_frame = df_id_start_stop[df_id_start_stop.id==matched_id].min_idx.values[0]
            matched_id_frame_data = pickle.load(open(f"{session_frame_dir}/{matched_id_min_frame}.pb","rb"))[1]
            matched_id_frame_data = [xr for xr in matched_id_frame_data if (xr['track_id']==matched_id)][0]
            matched_id_bb = matched_id_frame_data['bbox'][:4].astype(int)
            X_TL2, Y_TL2, X_BR2, Y_BR2 = matched_id_bb
            p1, p2, p3, p4  = map(Point, [[X_TL2,Y_TL2], [X_TL2,Y_BR2], [X_BR2,Y_BR2],[X_BR2, Y_TL2]]) 
            matched_id_polygon = Polygon(p1, p2, p3, p4)
            
            #find intersection of two polygons
            # check if intersection exists
            if id_polygon.encloses_point(matched_id_polygon.centroid) & matched_id_polygon.encloses_point(id_polygon.centroid):
                X_TL_in, X_BR_in = sorted([X_TL1,X_TL2, X_BR1, X_BR2])[1:3]
                Y_TL_in, Y_BR_in = sorted([Y_TL1,Y_TL2, Y_BR1, Y_BR2])[1:3]
                p1, p2, p3, p4  = map(Point, [[X_TL_in,Y_TL_in], [X_TL_in,Y_BR_in], [X_BR_in,Y_BR_in],[X_BR_in, Y_TL_in]]) 
                intersection = Polygon(p1, p2, p3, p4)            

                #find polygon overlap
                area_intersection = np.abs(intersection.area)
                area_union = np.abs(id_polygon.area) + np.abs(matched_id_polygon.area) - area_intersection
                overlap_fraction  = (area_intersection/area_union).evalf()
            else:
                overlap_fraction=0.
            
            if overlap_fraction > MAX_BBOX_OVERLAP:
                successful_matches.append(((row['id'], matched_id), overlap_fraction, (min(id_max_frame, matched_id_min_frame)-51, max(id_max_frame, matched_id_min_frame)+51)))
            
    if len(successful_matches) > 0:
        successful_match = sorted(successful_matches, key=lambda x: x[1])[-1]
        final_matches.append(successful_match)
        successful_matched_id = successful_match[0]
        print('\t Successful match:',successful_match)
        if row['id'] in final_id_maps.keys():
            final_id_maps[successful_matched_id] = final_id_maps[row['id']]
        else:
            final_id_maps[successful_matched_id] = row['id']
        direct_id_maps[successful_matched_id] = row['id']


Got Tracking Data...
Got ID start and stop from raw data...
	 Successful match: ((7, 3073), 0.484817001180638, (57531, 57666))
	 Successful match: ((36, 178), 0.866899485348178, (3108, 3297))
	 Successful match: ((41, 216), 0.665537017726799, (3666, 3879))
	 Successful match: ((67, 612), 0.919202518363064, (9366, 9504))
	 Successful match: ((124, 1534), 0.961660236796594, (25494, 25743))
	 Successful match: ((126, 1712), 0.610794345075373, (28431, 28542))
	 Successful match: ((146, 899), 0.810927541093593, (13902, 14043))
	 Successful match: ((178, 3540), 0.856039742966683, (63300, 63447))
	 Successful match: ((216, 325), 0.850140955145561, (4863, 5031))
	 Successful match: ((230, 340), 0.614650478151277, (4854, 5298))
	 Successful match: ((232, 403), 0.557807983336167, (5850, 5976))
	 Successful match: ((294, 393), 0.570062716547520, (5667, 5838))
	 Successful match: ((325, 766), 0.576697762715023, (11946, 12189))
	 Successful match: ((340, 514), 0.931577542485986, (7083, 8025))
	 Suc

In [ ]:
# for given frame ranges get video frames and tracking results
all_frames = mmcv.VideoReader(session_video_file)

required_frame_ids = np.unique(np.concatenate([np.arange(xr[2][0],xr[2][1]) for xr in final_matches]))
frame_data_dict = {}

for frame_idx, frame_img in enumerate(all_frames):
    if (frame_idx in required_frame_ids) & (frame_idx%3==0):
        frame_data_dict[frame_idx] = frame_img
    if frame_idx%10000==0:
        print(f"Looped {frame_idx} images")
        



[h264 @ 0x79a29c0] error while decoding MB 189 50, bytestream -13
[h264 @ 0x7d11740] error while decoding MB 189 50, bytestream -13
[h264 @ 0x9708440] error while decoding MB 234 84, bytestream -5
[h264 @ 0x7d11740] error while decoding MB 189 50, bytestream -13
[h264 @ 0x9708440] error while decoding MB 234 84, bytestream -5


Looped 0 images


[h264 @ 0x7dda500] error while decoding MB 238 62, bytestream -5
[h264 @ 0x7e3fb40] error while decoding MB 109 127, bytestream -5
[h264 @ 0x7dcb880] error while decoding MB 194 103, bytestream -5
[h264 @ 0x7e3fb40] error while decoding MB 105 39, bytestream -5
[h264 @ 0x7dda500] error while decoding MB 234 120, bytestream -5
[h264 @ 0x9708440] cabac decode of qscale diff failed at 177 30
[h264 @ 0x9708440] error while decoding MB 177 30, bytestream 380599
[h264 @ 0x7e17240] error while decoding MB 229 51, bytestream -19
[h264 @ 0x7dcb880] error while decoding MB 10 54, bytestream -5
[h264 @ 0x96f3fc0] cabac decode of qscale diff failed at 90 35
[h264 @ 0x96f3fc0] error while decoding MB 90 35, bytestream 225434
[h264 @ 0x7e17240] error while decoding MB 15 70, bytestream -7
[h264 @ 0x96dfb40] error while decoding MB 231 112, bytestream -5
[h264 @ 0x9708440] cabac decode of qscale diff failed at 24 90
[h264 @ 0x9708440] error while decoding MB 24 90, bytestream 220573
[h264 @ 0x7e53fc0

Looped 10000 images


[h264 @ 0x7cfbdc0] cabac decode of qscale diff failed at 64 20
[h264 @ 0x7cfbdc0] error while decoding MB 64 20, bytestream 62291
[h264 @ 0x7e53fc0] cabac decode of qscale diff failed at 122 41
[h264 @ 0x7e53fc0] error while decoding MB 122 41, bytestream 360573


In [45]:
# init tracking model to put tracking results on frame img
SOURCE_ROOT = '/home/prasoon/openmmlab/edusenseV2compute/compute/videoV3'
run_config = {
    'track_config':f'{SOURCE_ROOT}/configs/mmlab/ocsort_yolox_x_crowdhuman_mot17-private-half.py',
    'track_checkpoint':f'{SOURCE_ROOT}/models/mmlab/ocsort_yolox_x_crowdhuman_mot17-private-half_20220813_101618-fe150582.pth',
    'device':'cuda:1',
}


2023-07-12 05:15:52,809 - mmdet - INFO - image shape: height=800, width=1440 in YOLOX.__init__
2023-07-12 05:15:52,913 - mmtrack - INFO - initialize YOLOX with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth'}
2023-07-12 05:15:52,915 - mmcv - INFO - load model from: https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth
2023-07-12 05:15:52,916 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth
2023-07-12 05:15:55,082 - mmcv - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.multi_level_conv_cls.0.weight: copying a param with shape torch.Size([80, 320, 1, 1]) from checkpoint, the shape in current model is torch.Size([1, 320, 1, 

load checkpoint from local path: /home/prasoon/openmmlab/edusenseV2compute/compute/videoV3/models/mmlab/ocsort_yolox_x_crowdhuman_mot17-private-half_20220813_101618-fe150582.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: ema_detector_backbone_stem_conv_conv_weight, ema_detector_backbone_stem_conv_bn_weight, ema_detector_backbone_stem_conv_bn_bias, ema_detector_backbone_stem_conv_bn_running_mean, ema_detector_backbone_stem_conv_bn_running_var, ema_detector_backbone_stem_conv_bn_num_batches_tracked, ema_detector_backbone_stage1_0_conv_weight, ema_detector_backbone_stage1_0_bn_weight, ema_detector_backbone_stage1_0_bn_bias, ema_detector_backbone_stage1_0_bn_running_mean, ema_detector_backbone_stage1_0_bn_running_var, ema_detector_backbone_stage1_0_bn_num_batches_tracked, ema_detector_backbone_stage1_1_main_conv_conv_weight, ema_detector_backbone_stage1_1_main_conv_bn_weight, ema_detector_backbone_stage1_1_main_conv_bn_bias, ema_detector_back

In [77]:
# compile videos for each match
session_matching_cache_root = f'../cache/analysis_tracking/session_matching_info/{session}'
os.makedirs(session_matching_cache_root, exist_ok=True)

for final_match in final_matches:
    idA, idB = final_match[0]
    min_frame, max_frame = final_match[-1]
    match_cache_dir = f'{session_matching_cache_root}/{idA}_{idB}'
    match_video_file = f'{session_matching_cache_root}/{idA}_{idB}-{int(min_frame)//(15*60)}:{int(min_frame/15)%60}_{int(max_frame)//(15*60)}:{int(max_frame/15)%60}.mp4'
    if os.path.exists(match_video_file):
        continue
    print(match_video_file)
    os.makedirs(match_cache_dir, exist_ok=True)
    frame_num=0
    tracking_model = init_tracking_model(run_config['track_config'],
                                     run_config['track_checkpoint'],
                                     device=run_config['device'])
    for frame_idx in range(min_frame, max_frame):
        frame_tracking_file = f"{session_frame_dir}/{frame_idx}.pb"
        if os.path.exists(frame_tracking_file) and (frame_idx in frame_data_dict):
            frame_file_out = os.path.join(match_cache_dir, f'{frame_num:06d}.jpg')
            frame_num+=1
            if os.path.exists(frame_file_out):
                continue
            frame_tracking_results = pickle.load(open(frame_tracking_file,"rb"))[1]
            frame_img = frame_data_dict[frame_idx]
            frame_tracking_results = [xr for xr in frame_tracking_results if xr['track_id'] in (idA, idB)]
            if len(frame_tracking_results)>0:
                frame_tracking_results = {
                    'track_bboxes':[np.array([[xr['track_id']]+xr['bbox'].tolist() for xr in frame_tracking_results])],
                    'det_bboxes':[np.array([xr['bbox'].tolist() for xr in frame_tracking_results])]}
                frame_track_img = tracking_model.show_result(
                    frame_img,
                    frame_tracking_results,
                    score_thr=0.1,
                    show=False,
                    wait_time=int(1000. / 5),
                    out_file=frame_file_out,
                    backend='cv2')
            else:
                cv2.imwrite(frame_file_out,frame_img)
    
    #         break
    # break
    mmcv.frames2video(match_cache_dir, match_video_file, fps=5, fourcc='mp4v')
    del tracking_model
    print(f"Done for match {final_match}")
    
    

../cache/analysis_tracking/session_matching_info/classinsight-cmu_21127L_ph_a22_201903191630/6_297-11:14_11:21.mp4


2023-07-12 11:04:38,304 - mmdet - INFO - image shape: height=800, width=1440 in YOLOX.__init__
2023-07-12 11:04:38,365 - mmtrack - INFO - initialize YOLOX with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth'}
2023-07-12 11:04:38,367 - mmcv - INFO - load model from: https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth
2023-07-12 11:04:38,368 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth
2023-07-12 11:04:39,494 - mmcv - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.multi_level_conv_cls.0.weight: copying a param with shape torch.Size([80, 320, 1, 1]) from checkpoint, the shape in current model is torch.Size([1, 320, 1, 

load checkpoint from local path: /home/prasoon/openmmlab/edusenseV2compute/compute/videoV3/models/mmlab/ocsort_yolox_x_crowdhuman_mot17-private-half_20220813_101618-fe150582.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: ema_detector_backbone_stem_conv_conv_weight, ema_detector_backbone_stem_conv_bn_weight, ema_detector_backbone_stem_conv_bn_bias, ema_detector_backbone_stem_conv_bn_running_mean, ema_detector_backbone_stem_conv_bn_running_var, ema_detector_backbone_stem_conv_bn_num_batches_tracked, ema_detector_backbone_stage1_0_conv_weight, ema_detector_backbone_stage1_0_bn_weight, ema_detector_backbone_stage1_0_bn_bias, ema_detector_backbone_stage1_0_bn_running_mean, ema_detector_backbone_stage1_0_bn_running_var, ema_detector_backbone_stage1_0_bn_num_batches_tracked, ema_detector_backbone_stage1_1_main_conv_conv_weight, ema_detector_backbone_stage1_1_main_conv_bn_weight, ema_detector_backbone_stage1_1_main_conv_bn_bias, ema_detector_back

2023-07-12 11:05:00,948 - mmdet - INFO - image shape: height=800, width=1440 in YOLOX.__init__
2023-07-12 11:05:01,007 - mmtrack - INFO - initialize YOLOX with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth'}
2023-07-12 11:05:01,009 - mmcv - INFO - load model from: https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth
2023-07-12 11:05:01,010 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth
2023-07-12 11:05:01,489 - mmcv - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.multi_level_conv_cls.0.weight: copying a param with shape torch.Size([80, 320, 1, 1]) from checkpoint, the shape in current model is torch.Size([1, 320, 1, 

load checkpoint from local path: /home/prasoon/openmmlab/edusenseV2compute/compute/videoV3/models/mmlab/ocsort_yolox_x_crowdhuman_mot17-private-half_20220813_101618-fe150582.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: ema_detector_backbone_stem_conv_conv_weight, ema_detector_backbone_stem_conv_bn_weight, ema_detector_backbone_stem_conv_bn_bias, ema_detector_backbone_stem_conv_bn_running_mean, ema_detector_backbone_stem_conv_bn_running_var, ema_detector_backbone_stem_conv_bn_num_batches_tracked, ema_detector_backbone_stage1_0_conv_weight, ema_detector_backbone_stage1_0_bn_weight, ema_detector_backbone_stage1_0_bn_bias, ema_detector_backbone_stage1_0_bn_running_mean, ema_detector_backbone_stage1_0_bn_running_var, ema_detector_backbone_stage1_0_bn_num_batches_tracked, ema_detector_backbone_stage1_1_main_conv_conv_weight, ema_detector_backbone_stage1_1_main_conv_bn_weight, ema_detector_backbone_stage1_1_main_conv_bn_bias, ema_detector_back

...

load checkpoint from local path: /home/prasoon/openmmlab/edusenseV2compute/compute/videoV3/models/mmlab/ocsort_yolox_x_crowdhuman_mot17-private-half_20220813_101618-fe150582.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: ema_detector_backbone_stem_conv_conv_weight, ema_detector_backbone_stem_conv_bn_weight, ema_detector_backbone_stem_conv_bn_bias, ema_detector_backbone_stem_conv_bn_running_mean, ema_detector_backbone_stem_conv_bn_running_var, ema_detector_backbone_stem_conv_bn_num_batches_tracked, ema_detector_backbone_stage1_0_conv_weight, ema_detector_backbone_stage1_0_bn_weight, ema_detector_backbone_stage1_0_bn_bias, ema_detector_backbone_stage1_0_bn_running_mean, ema_detector_backbone_stage1_0_bn_running_var, ema_detector_backbone_stage1_0_bn_num_batches_tracked, ema_detector_backbone_stage1_1_main_conv_conv_weight, ema_detector_backbone_stage1_1_main_conv_bn_weight, ema_detector_backbone_stage1_1_main_conv_bn_bias, ema_detector_back

2023-07-12 11:16:07,913 - mmdet - INFO - image shape: height=800, width=1440 in YOLOX.__init__
2023-07-12 11:16:07,997 - mmtrack - INFO - initialize YOLOX with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth'}
2023-07-12 11:16:07,999 - mmcv - INFO - load model from: https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth
2023-07-12 11:16:08,004 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth
2023-07-12 11:16:08,499 - mmcv - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.multi_level_conv_cls.0.weight: copying a param with shape torch.Size([80, 320, 1, 1]) from checkpoint, the shape in current model is torch.Size([1, 320, 1, 

load checkpoint from local path: /home/prasoon/openmmlab/edusenseV2compute/compute/videoV3/models/mmlab/ocsort_yolox_x_crowdhuman_mot17-private-half_20220813_101618-fe150582.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: ema_detector_backbone_stem_conv_conv_weight, ema_detector_backbone_stem_conv_bn_weight, ema_detector_backbone_stem_conv_bn_bias, ema_detector_backbone_stem_conv_bn_running_mean, ema_detector_backbone_stem_conv_bn_running_var, ema_detector_backbone_stem_conv_bn_num_batches_tracked, ema_detector_backbone_stage1_0_conv_weight, ema_detector_backbone_stage1_0_bn_weight, ema_detector_backbone_stage1_0_bn_bias, ema_detector_backbone_stage1_0_bn_running_mean, ema_detector_backbone_stage1_0_bn_running_var, ema_detector_backbone_stage1_0_bn_num_batches_tracked, ema_detector_backbone_stage1_1_main_conv_conv_weight, ema_detector_backbone_stage1_1_main_conv_bn_weight, ema_detector_backbone_stage1_1_main_conv_bn_bias, ema_detector_back

2023-07-12 11:16:38,555 - mmdet - INFO - image shape: height=800, width=1440 in YOLOX.__init__
2023-07-12 11:16:38,616 - mmtrack - INFO - initialize YOLOX with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth'}
2023-07-12 11:16:38,618 - mmcv - INFO - load model from: https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth
2023-07-12 11:16:38,619 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth
2023-07-12 11:16:39,102 - mmcv - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.multi_level_conv_cls.0.weight: copying a param with shape torch.Size([80, 320, 1, 1]) from checkpoint, the shape in current model is torch.Size([1, 320, 1, 

load checkpoint from local path: /home/prasoon/openmmlab/edusenseV2compute/compute/videoV3/models/mmlab/ocsort_yolox_x_crowdhuman_mot17-private-half_20220813_101618-fe150582.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: ema_detector_backbone_stem_conv_conv_weight, ema_detector_backbone_stem_conv_bn_weight, ema_detector_backbone_stem_conv_bn_bias, ema_detector_backbone_stem_conv_bn_running_mean, ema_detector_backbone_stem_conv_bn_running_var, ema_detector_backbone_stem_conv_bn_num_batches_tracked, ema_detector_backbone_stage1_0_conv_weight, ema_detector_backbone_stage1_0_bn_weight, ema_detector_backbone_stage1_0_bn_bias, ema_detector_backbone_stage1_0_bn_running_mean, ema_detector_backbone_stage1_0_bn_running_var, ema_detector_backbone_stage1_0_bn_num_batches_tracked, ema_detector_backbone_stage1_1_main_conv_conv_weight, ema_detector_backbone_stage1_1_main_conv_bn_weight, ema_detector_backbone_stage1_1_main_conv_bn_bias, ema_detector_back

In [61]:
final_matches

[((6, 297), 0.595250966432411, (10110, 10218)),
 ((12, 580), 0.957698132256436, (17010, 17184)),
 ((13, 386), 0.409340138779289, (14061, 14175)),
 ((18, 1503), 0.699480390063350, (39888, 40017)),
 ((22, 146), 0.866924134319565, (2730, 2865)),
 ((26, 541), 0.573254039216133, (16497, 16617)),
 ((43, 502), 0.589797297297297, (16065, 16260)),
 ((70, 703), 0.910805575842251, (19449, 19698)),
 ((73, 162), 0.409940880227721, (3054, 3279)),
 ((109, 177), 0.628264906678780, (4170, 4308)),
 ((146, 563), 0.579197427837110, (16221, 16872)),
 ((177, 199), 0.423772888634721, (5496, 5718)),
 ((199, 579), 0.858752977203130, (17040, 17184)),
 ((386, 577), 0.512425336839990, (17046, 17154)),
 ((502, 807), 0.680899842384296, (21009, 21165)),
 ((577, 634), 0.632541219692219, (18051, 18156)),
 ((634, 826), 0.800397047913931, (20925, 21654)),
 ((703, 860), 0.910371130996039, (22329, 22650)),
 ((826, 946), 0.632298136645963, (24330, 24477)),
 ((860, 1343), 0.962441314553991, (37314, 37488)),
 ((1114, 1168), 

In [62]:
frame_tracking_results

[{'bbox': array([7.08472229e+02, 5.75314941e+02, 1.18627930e+03, 1.14788696e+03,
         9.46871281e-01]),
  'track_id': 60.0},
 {'bbox': array([2.07448242e+03, 8.00528442e+02, 2.68943823e+03, 1.68520776e+03,
         9.26436961e-01]),
  'track_id': 49.0},
 {'bbox': array([1.59178540e+03, 1.13098840e+03, 2.23899561e+03, 1.81847559e+03,
         9.24214184e-01]),
  'track_id': 5.0},
 {'bbox': array([2.67681177e+03, 3.59493958e+02, 3.01993066e+03, 9.31043152e+02,
         9.21242714e-01]),
  'track_id': 10.0},
 {'bbox': array([1.96425842e+03, 4.54412537e+02, 2.38289478e+03, 1.04466431e+03,
         9.17824984e-01]),
  'track_id': 2.0},
 {'bbox': array([4.64326813e+02, 8.88158386e+02, 9.81837036e+02, 1.53158704e+03,
         9.05112624e-01]),
  'track_id': 128.0},
 {'bbox': array([9.45054871e+02, 1.35933911e+03, 1.65860999e+03, 2.22343286e+03,
         9.00693417e-01]),
  'track_id': 13.0},
 {'bbox': array([9.21566772e+02, 4.74281189e+02, 1.28422644e+03, 1.11686230e+03,
         8.883643

In [64]:
results_track_df = np.array([[xr['track_id']]+xr['bbox'].tolist() for xr in frame_tracking_results])
results_det_df = np.array([xr['bbox'].tolist() for xr in frame_tracking_results])
results_det_df

array([[7.08472229e+02, 5.75314941e+02, 1.18627930e+03, 1.14788696e+03,
        9.46871281e-01],
       [2.07448242e+03, 8.00528442e+02, 2.68943823e+03, 1.68520776e+03,
        9.26436961e-01],
       [1.59178540e+03, 1.13098840e+03, 2.23899561e+03, 1.81847559e+03,
        9.24214184e-01],
       [2.67681177e+03, 3.59493958e+02, 3.01993066e+03, 9.31043152e+02,
        9.21242714e-01],
       [1.96425842e+03, 4.54412537e+02, 2.38289478e+03, 1.04466431e+03,
        9.17824984e-01],
       [4.64326813e+02, 8.88158386e+02, 9.81837036e+02, 1.53158704e+03,
        9.05112624e-01],
       [9.45054871e+02, 1.35933911e+03, 1.65860999e+03, 2.22343286e+03,
        9.00693417e-01],
       [9.21566772e+02, 4.74281189e+02, 1.28422644e+03, 1.11686230e+03,
        8.88364315e-01],
       [3.00766602e+03, 5.61544434e+02, 3.33725854e+03, 1.08086670e+03,
        8.81299555e-01],
       [2.46823120e+03, 5.94878723e+02, 2.85415015e+03, 1.28993457e+03,
        8.69975626e-01],
       [2.80915796e+03, 8.6588